In [1]:
import pandas as pd
import numpy as np
import requests
import json
import sqlalchemy as sql
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#user imput will be used for variable coin_id from a list in a widget, for not I set the variable value
coin_id = "bitcoin"

start_time = 1641042000 #2022-01-01 1pmGMT
end_time = 1668208735 #now
url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range?vs_currency=usd&from={start_time}&to={end_time}"
response = requests.get(url)
data = response.json()

timestamp_list = []
price_list = []

for row in data['prices'] :
    timestamp_list.append(datetime.datetime.fromtimestamp(row[0]/1000))
    price_list.append(row[1])
    
raw_data = {'timestamp': timestamp_list, 'price' : price_list}
df = pd.DataFrame(raw_data, columns=['timestamp','price'])
pd.options.display.float_format = '{:.2f}'.format
df = df.set_index('timestamp')
df

,price
timestamp,
2022-01-01 16:00:00,47816.08
2022-01-02 16:00:00,47387.21
2022-01-03 16:00:00,46531.14
2022-01-04 16:00:00,45938.02
2022-01-05 16:00:00,43647.15
...,...
2022-11-06 16:00:00,20907.00
2022-11-07 16:00:00,20597.76
2022-11-08 16:00:00,18562.35


In [3]:
#slices the data frame to about 90 days at time of testing. this step can be fixed with datetime parameters
df_slice90 = df.loc['2022-08-11 16:00:00' : '2022-11-11 16:00:00']
df_slice90

,price
timestamp,
2022-08-11 17:00:00,23948.75
2022-08-12 17:00:00,24411.38
2022-08-13 17:00:00,24433.65
2022-08-14 17:00:00,24312.54
2022-08-15 17:00:00,24179.01
...,...
2022-11-06 16:00:00,20907.00
2022-11-07 16:00:00,20597.76
2022-11-08 16:00:00,18562.35


In [ ]:
#df_slice90.plot(figsize=(10,7),title=f"Daily Price of {coin_id} over past 90 days")

In [4]:
coin_price_df = df_slice90 

def sma_plot(coin_price_df):  #Simple Moving Average
    sma_df = coin_price_df
     
    rolling_average = sma_df.rolling(7).mean()
    sma_df["SMA7"] = rolling_average
    #sma_df["SMA7"] = sma_df.rolling(7).mean()
    #sma_df["SMA20"] = sma_df.loc[:, "price"].rolling(20).mean()
   

    sma_df.dropna(inplace=True)

    return sma_df, rolling_average
sma_plot(coin_price_df)

/Users/evieair/opt/anaconda3/envs/testing/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/evieair/opt/anaconda3/envs/testing/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(                       price     SMA7
 timestamp                            
 2022-08-17 17:00:00 23359.42 24079.73
 2022-08-18 17:00:00 23248.38 23979.68
 2022-08-19 17:00:00 20945.83 23484.60
 2022-08-20 17:00:00 21175.23 23019.11
 2022-08-21 17:00:00 21615.76 22633.85
 ...                      ...      ...
 2022-11-06 16:00:00 20907.00 20671.21
 2022-11-07 16:00:00 20597.76 20685.94
 2022-11-08 16:00:00 18562.35 20410.61
 2022-11-09 16:00:00 15742.44 19779.11
 2022-11-10 16:00:00 17594.92 19405.40
 
 [86 rows x 2 columns],
                        price
 timestamp                   
 2022-08-11 17:00:00      NaN
 2022-08-12 17:00:00      NaN
 2022-08-13 17:00:00      NaN
 2022-08-14 17:00:00      NaN
 2022-08-15 17:00:00      NaN
 ...                      ...
 2022-11-06 16:00:00 20671.21
 2022-11-07 16:00:00 20685.94
 2022-11-08 16:00:00 20410.61
 2022-11-09 16:00:00 19779.11
 2022-11-10 16:00:00 19405.40
 
 [92 rows x 1 columns])

In [ ]:
#plot of price with Simple Moving Averages showing how longer timeframe of average smooths out curve
df_slice90.plot(figsize=(10,7), title=f"{coin_id} Daily Price with the Simple moving Average for 7,12, and 30 Days")

In [ ]:
#text significance of Simple Moving Average
print('''
The Simple Moving Average (SMA) is calculated by taking the average price during our time period of analaysis (for this case is 90 days)
and dividing by the number of price data points we have during that time.  It gives you an idea of how the coin is performing in general
over the time period without indicating all the highs and lows.  As you can see the longer the period of our average the smoother
the line.  Because crypto markets are active 24/7 and there is so much activity. Using a shorter timeframe for a moving average
gives you a more accurate picture of coin performance.
'''
      
      

In [ ]:
#calculates Weighted Moving Average aka Exponential Moving Average over 7 days
df_slice90["WMA7"] = df_slice90["price"].ewm(7).mean()
df_slice90.dropna(inplace=True)
df_slice90.head()

In [ ]:
#creates new df to plot just price and 7 day sma & wma
df_wma = df_slice90[["price", "SMA7", "WMA7"]]
df_wma.head()

In [ ]:
df_wma.plot(figsize=(10,7), title=f"{coin_id} Daily Price vs. 7-Day Simple Moving Average and  7-Day Weighted Moving Average")

In [ ]:
#text significance of Weighted Moving Average
print('''
The Weighted Moving Average (WMA), also called the Exponential Moving Average, uses the SMA in its calculations, but
gives more weight to recent prices in an attempt to make data more responsive to new information. You see in the plot
the green line indicating WMA over 7 days tracks closely to the simple moving average for the first 2/3 of 90 days, but 
then responds quicker to the change in price during the last 1/3 of the 90 days.
'''
      

In [ ]:
#functions for calculating the bollinger bands
prices = df_slice90["price"]
rate = 7
prices

def get_sma(prices, rate):
    return prices.rolling(rate).mean()

def get_bollinger_bands(prices, rate=20):
    sma = get_sma(prices, rate) # <-- Get SMA for 20 days
    std = prices.rolling(rate).std() # <-- Get rolling standard deviation for 20 days
    
    bollinger_up = sma + std * 2 # Calculate top band
    bollinger_down = sma - std * 2 # Calculate bottom band
    return bollinger_up, bollinger_down

symbol = coin_id
#df = pdr.DataReader(symbol, 'yahoo', '2014-07-01', '2015-07-01')
#df.index = np.arange(df.shape[0])
closing_prices = df['price']

bollinger_up, bollinger_down = get_bollinger_bands(closing_prices)

In [ ]:
#plots the bollinger bands aling with the price
plt.title(symbol + ' Bollinger Bands for past 90 Days')
plt.xlabel('timestamp')
plt.ylabel('Daily Prices')
plt.plot(closing_prices, label='Daily Price')
plt.plot(bollinger_up, label='Bollinger Up', c='g')
plt.plot(bollinger_down, label='Bollinger Down', c='r')
plt.legend()
plt.xticks(rotation = 45)
plt.show()


In [ ]:
#text significance of Bollinger Bands
print('''
The Bolliger Bands are often used as indicators of buy/sell opportunities. This technical indicator allows traders
to analyze the volatility of a coin and whether the price is high or low on a relative basis.
The top band is typically two standard deviations above the SMA and the bottom band is typically two standard deviations
below the SMA. This means 95% of the high/low price variation fits between these two bands. You can see how they 
form ribbons around the price.
Areas where the blue price line touches or goes below the red, Bollinger low band, are times you would want to consider
buying opportunities, because the price is lower than 95% of it's usual performance.
Areas where the blue, price line touches or goes above the green, Bollinger hight band, are times you would want to 
consider selling opportunities, because the price is higher than 95% of it's usual performance
'''